In [1]:
import string
import nltk
import re
import sys
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from glob import glob
import zipfile
import os
import numpy as np
from tqdm import tqdm
import concurrent.futures
import multiprocessing
num_processes = multiprocessing.cpu_count()

plt.rcParams['figure.figsize'] = [17, 10]

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from script import read_instances as r

In [2]:
path = "/home/bamrung/Documents/metaheuristique/template-jobshop/instances"

Duration = []
Instances = []

files = r.get_files(path)[1:]
for f in files:
    # read one instance and output a data matrix
    data = r.read_instance(f)
#     data = read(f)
    
    print(f)
    print(data)
#     machines = r.get_machines(data)
#     duration = r.get_duration(data)

/home/bamrung/Documents/metaheuristique/template-jobshop/instances/aaa1
[['0' '3' '1' '3' '2' '2']
 ['1' '2' '0' '2' '2' '4']]
/home/bamrung/Documents/metaheuristique/template-jobshop/instances/abz5
[['4' '88' '8' '68' '6' '94' '5' '99' '1' '67' '2' '89' '9' '77' '7' '99'
  '0' '86' '3' '92']
 ['5' '72' '3' '50' '6' '69' '4' '75' '2' '94' '8' '66' '0' '92' '1' '82'
  '7' '94' '9' '63']
 ['9' '83' '8' '61' '0' '83' '1' '65' '6' '64' '5' '85' '7' '78' '4' '85'
  '2' '55' '3' '77']
 ['7' '94' '2' '68' '1' '61' '4' '99' '3' '54' '6' '75' '5' '66' '0' '76'
  '9' '63' '8' '67']
 ['3' '69' '4' '88' '9' '82' '8' '95' '0' '99' '2' '67' '6' '95' '5' '68'
  '7' '67' '1' '86']
 ['1' '99' '4' '81' '5' '64' '6' '66' '8' '80' '2' '80' '7' '69' '9' '62'
  '3' '79' '0' '88']
 ['7' '50' '1' '86' '4' '97' '3' '96' '0' '95' '8' '97' '2' '66' '5' '99'
  '6' '52' '9' '71']
 ['4' '98' '6' '73' '3' '82' '2' '51' '1' '71' '5' '94' '7' '85' '0' '62'
  '8' '95' '9' '79']
 ['0' '94' '6' '71' '3' '81' '7' '85' '1'

/home/bamrung/Documents/metaheuristique/template-jobshop/instances/ta75
[['15' '64' '1' ... '20' '5' '56']
 ['14' '62' '9' ... '83' '17' '42']
 ['4' '84' '9' ... '45' '0' '68']
 ...
 ['2' '84' '3' ... '8' '6' '17']
 ['12' '28' '14' ... '78' '8' '20']
 ['1' '51' '11' ... '57' '10' '79']]
/home/bamrung/Documents/metaheuristique/template-jobshop/instances/ta76
[['3' '52' '2' ... '66' '19' '49']
 ['12' '42' '11' ... '44' '8' '39']
 ['12' '10' '19' ... '73' '9' '14']
 ...
 ['9' '99' '15' ... '91' '17' '84']
 ['19' '94' '4' ... '8' '11' '78']
 ['4' '79' '11' ... '14' '18' '60']]
/home/bamrung/Documents/metaheuristique/template-jobshop/instances/ta77
[['2' '28' '18' ... '16' '13' '97']
 ['7' '78' '2' ... '73' '6' '80']
 ['1' '20' '18' ... '6' '16' '56']
 ...
 ['12' '70' '0' ... '25' '4' '78']
 ['14' '64' '15' ... '96' '2' '45']
 ['7' '43' '18' ... '52' '5' '64']]
/home/bamrung/Documents/metaheuristique/template-jobshop/instances/ta78
[['6' '97' '10' ... '30' '1' '50']
 ['5' '64' '1' ... '27' 

In [9]:
import numpy as np
from docplex.cp.model import CpoModel
import docplex
from os import listdir
from os.path import isfile, join
import numpy as np



# Get the instances path
# Change the path if neeeded


def solver(machines, duration):
    """Solve a scheduling problem using what was done in the lab

    Args:
        machines (np.ndarray): machines np array from get_machines()
        duration (np.ndarray): duration np array from get_duration()

    Returns:
        solution and nb_instance: solution of the scheduling problem with the number of instances
    """

    print(machines[0])
    upper_bound = int(duration.sum())
    print(upper_bound)
    nb_instance = 0
    # Par machine et par job
    mdl = CpoModel(name="tp4")
    per_machines = [[] for _ in range(len(duration[0]))]
    per_jobs = [[] for _ in duration]
    for i, job in enumerate(duration):
        for j, task in enumerate(job):
            v = mdl.interval_var(
                start=(0, upper_bound),
                end=(0, upper_bound),
                length=int(task),
                name=f"it_{i}_{j}",
            )
            per_machines[int(machines[i][j])].append(v)
            per_jobs[i].append(v)
            nb_instance = nb_instance + 1

    # Before start
    for i in per_jobs:
        for cpt in range(len(i) - 1):
            mdl.add(mdl.end_before_start(i[cpt], i[cpt + 1]))

    # No overlap

    for i in np.array(per_machines).T:
        mdl.add(mdl.no_overlap(i))

    # Makespan
    makespan = mdl.max([mdl.end_of(jobit[-1]) for jobit in per_jobs])
    mdl.add(mdl.minimize(makespan))

    # Solve
    print("\nSolving model....")
    msol = mdl.solve(TimeLimit=10)
    print("done")
    msol.print_solution()
    print("\n")
    solution = msol.get_objective_bounds()[0]
    print(
        "Le temps de traitement des tâches est de : ",
        solution,
        " et le nombre d'instanciation est de : ",
        nb_instance,
    )
    return solution, nb_instance

In [ ]:
#INSA
from config import setup
setup()

In [ ]:
# Local
from docplex.cp.config import context

context.solver.agent = "local"
context.solver.local.execfile = "/home/prembamrung/ibm/ILOG/CPLEX_Studio129/cpoptimizer/bin/x86-64_linux/cpoptimizer"

In [4]:
data=r.read_instance(files[0])
data

array([['0', '3', '1', '3', '2', '2'],
       ['1', '2', '0', '2', '2', '4']], dtype='<U1')

In [12]:

Duration = []
Instances = []



# read one instance and output a data matrix
data = r.read_instance(files[1])
# Create machine and duration matrixes here on the varible data
machines = r.get_machines(data)
duration = r.get_duration(data)
# apply model
solution, nb_instance = solver(machines, duration)
Duration.append(solution)
Instances.append(nb_instance)

[4 8 6 5 1 2 9 7 0 3]
7773

Solving model....
done
-------------------------------------------------------------------------------
Model constraints: 100, variables: integer: 0, interval: 100, sequence: 10
Solve status: Optimal, Fail status: SearchHasFailedNormally
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.01 sec
-------------------------------------------------------------------------------
Objective values: (859,), bounds: (859,), gaps: (0,)
it_0_0: (start=0, end=88, size=88, length=88)
it_0_1: (start=88, end=156, size=68, length=68)
it_0_2: (start=156, end=250, size=94, length=94)
it_0_3: (start=250, end=349, size=99, length=99)
it_0_4: (start=349, end=416, size=67, length=67)
it_0_5: (start=416, end=505, size=89, length=89)
it_0_6: (start=505, end=582, size=77, length=77)
it_0_7: (start=582, end=681, size=99, length=99)
it_0_8: (start=681, end=767, size=86, length=86)
it_0_9: (start=767, end=859, size=92, length=92)
it_1_0: (start=0, end=72, 